# Byte Management Classes and Functions 

In [ ]:
# libraries
import string
import numpy as np
import math
import encode

In [2]:
# load the file, remove bits from the byte array
# byte_store = loaded encoded text
# byte = int from 0 to 255, which holds the current byte, with used bits set to 0
# jj = index of byte_store (our current byte)
# ii = current bit within our current byte, from 0 (all available) to 7 (only the last bit is available)
# ll = len(byte_store)
class byte_mng:
    def __init__(self):
        self.byte_store = b'' # here I am filling the bytes, should be integers from 0 to 255
        self.byte = 0         # active byte

        self.ii = 0           # bits already read, goes from 0 to 7
        self.jj = 0           # bytes already read
        self.ll = 0           # length of the bytearray

        self.iio = 0
        self.jjo = 0
        self.rem = self.ll
        
    # unlike the previous version, the file is read first, not last
    # I need to check that this does what I think it does
    def load_file(self, filename):
        with open(filename, "rb") as f:
            self.byte_store = f.read() # the file should be put into a byte stream
            
        self.byte = self.byte_store[0]
        self.ii = 0         
        self.jj = 0          
        self.ll = len(self.byte_store) 

        self.iio = 0
        self.jjo = 0
        self.byteo = self.byte
        self.rem = self.ll
        
    # Once I open a file, I can start reading it. I never alter the data, but I will not access what has already been read. 
    # The first function assumes a known number of bits. We remove them one-by-one or 2 by 2 if we don't know how many we need.
    # sets self.iio and self.jjo, so the start of reading can be reset
    # return: value (int), whether or not we reach or superceed the length of the file
    def read_bits(self, lngth):
        done = False
        self.iio = self.ii
        self.jjo = self.jj
        self.byteo = self.byte
        self.rem = self.ll - self.jj
        
        # error-checking
        if self.ll == 0:
            print('File not defined.')
            done = True

        if self.rem == 0:
            print('End of file previously reached.')
            done = True

        ret_arr = [self.byte] # start with our return here
        used_bits = 8 - self.ii # used for length > remaining bit size
        k = (lngth - used_bits) // 8 # used for length > remaining bit size + 8

        #print("part1: ", self.ii, ", ", self.byte, ", ", lngth)
        
        # don't remove the bytes which don't need to be removed
        if (self.ii < (8 - lngth)) & (done == False): 
            mask = int(255) - int((2**(8 - lngth - self.ii) - 1))
            ret_arr[0] = ret_arr[0] & mask
            self.ii = self.ii + lngth
            mask = int(2**(8-self.ii) - 1)
            self.byte = self.byte & mask
            done = True

        #print("part1: ", self.ii, ", ", self.byte, ", ", lngth, ", ", done)

        #print(self.ii, self.jj)

        # full bytes
        if (done == False):
            if k > 0:
                for i in range(1, k+1):
                    if self.jj + i < self.ll:
                        ret_arr.append(self.byte_store[self.jj+i])
                    else:
                        print('Reached end of array.')
                        done = True
                        break
            if k >= 0:
                self.jj += k+1
                if self.jj < self.ll:
                    self.byte = self.byte_store[self.jj]
                else:
                    self.byte = 0 # end of array
                self.ii = 0 # return to beginning
                used_bits = used_bits + 8 * k
                
        if (done == False) & (lngth == used_bits):
            done = True
        elif (done == False) & (self.jj >= self.ll):
            print('Reached end of file')
            done = True
        elif (done == False) & (used_bits > lngth):
            print(lngth, used_bits)
            done = True

        #print(self.ii, self.jj)

        # and for the last byte
        if done == False:
            ret_arr.append(self.byte)
            #remaining bits
            rem_bits = lngth - used_bits
            #print("part2: ", rem_bits, ", ", lngth, ", ", used_bits)
            i = rem_bits
            mask = int(255) - int((2**(8-i) - 1))
            #if i > 8: print(i)
            ret_arr[-1] = ret_arr[-1] & mask
            self.ii = i
            mask = int((2**(8-i) - 1))
            self.byte = self.byte & mask

        #print(self.ii, self.jj, ret_arr)

        return self.byte_to_int(ret_arr, self.iio, self.ii)

    # this combines all bits into an integer
    def byte_to_int(self, byte_arr, start, end):
        lgt = len(byte_arr)
        num_bits = (lgt-1)*8 - start + end
        if end == 0:
            num_bits += 8
        out = 0
        for i in range(num_bits):
            # we want to take of the bit at index start + i
            ind = start + i
            kk = ind // 8
            ind = ind % 8
            pwr1 = int(2**(7-ind))
            num = pwr1 & byte_arr[kk] 
            #if ind < 7: num = num >> (7-ind)
            pwr_full = int(2**(num_bits - i - 1))
            if num > 0: out += pwr_full
        return(out)

    # if I read the next byte as a test, and find it is for the next section instead
    def revert(self):
        self.ii = self.iio
        self.jj = self.jjo
        self.byte = self.byteo
        self.rem = self.ll - self.jj
        
    def to_end_of_byte(self):
        if self.ii > 0:
            self.jj = self.jj + 1
        self.ii = 0
        self.rem = self.ll - self.jj
        if self.rem > 0:
            self.byte = self.byte_store[self.jj]
        else:
            self.byte = 0

    # now for the code for ternary code
    def read_ternary(self):
        # 00, 01, 10, 11
        # leading will be 1 or 0

        # 1 for 2 and 0 for 1
        # this gets the leading digit
        ter = [self.read_bits(1)+1]

        # repeat for the rest of the digits
        # 01 = 0, 10 = 1, 11 = 2
        code = 0
        while code >= 0:
            code = self.read_bits(2) - 1
            if code >= 0:
                ter.append(code)

        #print(ter, code)

        # now to return the number
        num = 0
        for i in range(len(ter)):
            #print(num)
            num += ter[i] * (3**(len(ter)-1-i))
            
        return num

    def read_list(self):
        word_arr = []
        cont = True
        while cont:
            word = self.read_word()
            if word == '':
                cont = False
            else:
                word_arr.append(word)
        return word_arr

    def read_word(self):
        word = ''
        cont = True
        while cont:
            char = self.read_char()
            if char == '':
                cont = False
            else:
                word = word + char
        return word

    def read_char(self):
        #000xx = stop
        #0010x = stop
        #0011x = ab
        #01xxx = c-j
        #10xxx = k-r
        #11xxx = s-z
        
        code_a = ord('a') - int('110', 2)
        #num = ord(cha) - ord('a') + code_a # the order is unchanged, but the numbering starts earlier
        #ord(cha) = num + ord('a') - code_a
        
        f3b = self.read_bits(3)
        if f3b == 0:
            char = ''
        elif f3b == 1:
            b4 = self.read_bits(1)
            if b4 == 0:
                char = '' # I could add a separate behavor here, but right now I have not.
            else:
                b5 = self.read_bits(1)
                if b5 == 0:
                    char = 'a'
                else:
                    char = 'b'
        else:
            b45 = self.read_bits(2)
            char_num = b45 + f3b*4 + code_a
            char = chr(char_num)

        return char

In [ ]:
byte_count = byte_mng()
byte_count.load_file("count_enc_test")

In [44]:
print(byte_count.ll)

90640


In [45]:
low = []
upp = []
for i in range(26):
    low.append(string.ascii_lowercase[i])
    upp.append(string.ascii_uppercase[i])
num = []
for i in range(10):
    num.append(string.digits[i])
sim = []

# Arithmetic Decoder Classes and Objects

In [6]:
class count_mng:
    # defines three modes 
    # 0 means static counts,
    # 1 means dynamic, fixed alphabet
    # 2 means dynamic alphabet
    # new_mode defines the count of the letter 'new' in the dynamic alphabet
    def __init__(self,alphabet_in=None, counts_in=None, new_mode=2):
        self.mode = 0
        if alphabet_in == None:
            self.mode = 2
            self.alphabet = []
            self.counts = []
        elif counts_in == None:
            self.mode = 1
            self.alphabet = alphabet_in
            self.counts = [1]*len(alphabet_in)
        else:
            self.mode = 0
            self.alphabet = alphabet_in
            self.counts = counts_in
        self.new_mode = new_mode

    def increment_count(self, i):
        if len(self.alphabet)==0:
            self.add_new_count(0)
        elif i < len(self.alphabet):
            self.counts[i] +=1
        else:
            self.add_new_count(i)
        return self.alphabet[i]

    #update
    def get_counts(self):
        if self.mode == 0: return self.get_counts_0()
        elif self.mode == 1: return self.get_counts_0()
        else: return self.get_counts_2()

    def get_counts_0(self):
        return self.counts.copy()
    
    # this is the most complicated case
    # I need to find the structure here and add it
    def get_counts_2(self):
        if len(self.alphabet) == 0:
            return None
        else:
            ret_arr = self.counts.copy()
            ret_arr.append(self.get_count_new())
            #print(ret_arr)
            return ret_arr
        
    # both of the following are for dynamic alphabets
    def get_count_new(self):
        modes = [1, 
                 max(1,len(self.alphabet)), 
                 max(1,len(self.alphabet)), 
                 max(1.0, len(self.alphabet)/2)]
        return modes[self.new_mode]
    
    def add_new_count(self, char):
        self.alphabet.append(char)
        modes = [1,
                 0,
                 1,
                 0.5]
        self.counts.append(modes[self.new_mode])

In [ ]:
# - keep track of and update l, h, bits
# {l_old, h_old, bits_old -> a, b -> h-l+1, h-l+1 / 2^n -> a+ term, b+ term -> l, 
#  h -> bin l, bin h -> enc update -> new bin l, new bin h, bits_new -> new l, new h}
class arith_code_mng:
    def __init__(self, counts_mngr, byte_m = byte_count):
        # window size
        self.N = 40
        self.win_size = 2**self.N

        # left and right borders
        # I know I don't need so many, but it should be fine right now
        self.l = 0
        self.h = self.win_size - 1
        self.bits = 0
        self.lo = 0
        self.ho = self.win_size - 1
        self.ln = 0
        self.hn = self.win_size - 1

        # counts
        self.find_counts = counts_mngr

        # encoding
        self.byte_encode = byte_m
        self.duration_static = 0
        self.static_count = 0
        self.end = 0
        self.window = 0

        self.byte_l = self.byte_encode.ll -  1
        self.cur_by = self.byte_encode.jj
        self.cur_bi = self.byte_encode.ii
        self.find_first_byte() # finds my window, removing it from the bit storage


    def read_bit(self):
        if (self.cur_by < self.byte_l):
            bit = self.byte_encode.read_bits(1)
            self.cur_by = self.byte_encode.jj
            self.cur_bi = self.byte_encode.ii                                                                
        else:
            bit = 0
            self.end = self.end + 1 # I don't think I need this, but ...
        return bit

    def find_first_byte(self):
        byte = 0
        for i in range(self.N):
            bit = self.read_bit()
            byte = byte << 1
            byte = byte | bit
        self.window = byte

    def shift_byte(self, n=1):
        for i in range(n):
            self.window = (self.window << 1) % self.win_size
            self.window = self.window | self.read_bit()

    def shift_byte_bits(self, n=1):
        for i in range(n):
            fb = (self.window // 2**(self.N-1)) << (self.N-1)
            self.window = (self.window << 1) % 2**(self.N-1) # removes 2 bits from the start, clearing a spot at the end and at the beginning
            self.window = self.window | self.read_bit() # add a bit to the end
            self.window = self.window | fb # return the first bit to the beginning

    # ----- here we have initialized everything,
    #      can check if we need to keep going
    #      and can find our byte string to read -----
            
    def remove_character(self):
        # find the location in the probability space for our byte array window 
        hl1_N = float(self.ho-self.lo+1) / float(self.win_size)
        w_pl = (self.window - self.lo) / hl1_N
        #print('w_pl, lo, ho, win')
        #print(bin(math.ceil(w_pl)), bin(self.lo), bin(self.ho), bin(self.window))

        # find the probabilities of various characters
        #char_arr = self.find_counts.alphabet.copy()
        a, char, b = self.find_window_chars(w_pl)

        # update l and h like before
        a_pl = math.ceil(hl1_N * a)
        b_pl = math.ceil(hl1_N * (b + 1)) - 1

        self.l = self.lo + a_pl
        self.h = self.lo + b_pl
        
        #print(format(self.l, '032b'),",", format(self.h, '032b'),",", format(self.window, '032b'))
        #print("new l, h: ", bin(self.h), bin(self.l))
        enc_n, bit_n = self.update_window() # shifts the window and records how many bits need to be removed from l and h
        self.new_lh(enc_n, bit_n) # removes bits from l and h
        #print("remove: ", bin(self.hn), bin(self.ln), enc_n, bit_n)

        # updates for the next run
        # there is definately an error here in the encoding ...
        self.char = char
        self.lo = self.ln
        self.ho = self.hn
        #self.update_context() # so that the next context uses the current character and removes the oldest

        return char #ch[char] # this array should not be global ... 

    #      a, char, b = self.find_windows_chars(w_pl, ind1, ind2, char_arr)
    def find_window_chars(self, w_pl):
        # this gets the counts
        char_counts = self.find_counts.get_counts()
        #print(char_counts)
        if char_counts is None:
            a = 0
            b = self.win_size - 1
            i = 0
        else:
            den = float(sum(char_counts))
            b_fl = float(0)
            a = lb = 0
            num = float(0)
            i = b = rb = -1
            while rb < math.ceil(w_pl):
                a = lb
                i += 1
                num = num + float(char_counts[i])
                b_fl = num / den
                rb = math.ceil(b_fl * self.win_size) - 1
                b = rb
                lb = rb + 1 # for the next cycle
        
        ret_char = self.find_counts.increment_count(i)

        return a, ret_char, b


    #enc_n, bit_n = self.update_window() # shifts the window and records how many bits need to be removed from l and h
    #self.new_lh(enc_n, bit_n) # removes bits from l and h
    # This is from the previous
    # This loads the bits into the array and updates bits, but so far doesn't update l and h
    def update_window(self):
        l = self.l
        h = self.h

        nbitload = 0
        nbits = 0

        #print(self.h-self.l)
        # both l and h are N bits long
        x = self.win_size
        #print(l / x, h / x)

        lb = 0
        hb = 0
        
        for i in range(self.N):
            #print("bits:", i, lb,hb, l/x, h/x)
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            #print("bits:", i, lb,hb, l/x, h/x, self.bits)
            if lb == hb:
                self.shift_byte() # we are no longer loading
                while self.bits > 0:
                    #self.byte_encode.load_bits(1,1) # they are removed earlier
                    #self.shift_byte_bits()
                    self.bits -= 1
                nbitload += 1
            else:
                #print(i)
                break

        # the final 0s in the file
        if nbitload == 0:
            self.duration_static += 1
        else:
            self.duration_static = 0

        # widening the window
        # print(l / x, h / x)
        for i in range(self.N - nbitload - 1):
            x = x // 2
            lb = l // x
            l = l % x
            hb = h // x
            h = h % x
            if lb == 1 and hb == 0:
                nbits += 1
                self.bits += 1
                self.shift_byte_bits()
            else:
                break

        #print(self.h-self.l, nbitload, nbits)
        return nbitload,nbits

    # again, I need to check this and check the logic
    # should be unchanged
    def new_lh(self, nbitload, nbits):
        # removing bits
        self.ln = (self.l << nbitload) % self.win_size
        self.hn = (self.h << nbitload) % self.win_size
        self.hn = self.hn | (2**nbitload - 1) # fill the end with 1s, not 0s

        # widening of the window
        if nbits > 0:
            l_fb = (self.ln // 2**(self.N-1)) << (self.N-1)
            h_fb = (self.hn // 2**(self.N-1)) << (self.N-1)
            self.ln = (self.ln % 2**(self.N-1-nbits)) << nbits
            self.hn = (self.hn % 2**(self.N-1-nbits)) << nbits
            self.hn = self.hn | (2**nbits - 1) # fill the end with 1s, not 0s
            # and return the first bit to the array
            self.ln = self.ln | l_fb
            self.hn = self.hn | h_fb
    
    # This deals with the widening of the window removing needed bits from my byte stream 
    def end_stream(self):
        self.byte_encode.ii = self.byte_encode.ii - self.bits
        while self.byte_encode.ii < 0:
            self.byte_encode.jj -= 1
            self.byte_encode.ii += 8
        self.byte_encode.byte = self.byte_encode.byte_store[self.byte_encode.jj]
        self.byte_encode.byte = self.byte_encode.byte % 2**(8-self.byte_encode.ii)

# Finding the Character Array



 - p1: {10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
 - size: (bit size -> 7, 6, 6, 6, {5}, 5, ..., 5, {4}, 4, 4, 4, 4, {2}, 2 or just all 7)
 - p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
 - p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - x = 61
 - p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
 - y = 61
 - p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
 - z = 61, to indicate end of file 
 - count4

Indicies encoded 0-60, 61 indicates need to increment the first index only if not obvious.

In [ ]:
#byte_count.ii = 0
#byte_count.jj = 0
#byte_count.byte = byte_count.byte_store[0]

In [8]:
# first = p1
# Encoding unchanged (not arithmetic)

def remove_sym_arr(bm):
    sym_space = []
    bit_len = int(7)
    rem_chars = int(128 - 26*2 - 10)
    while rem_chars > 0:
        sym_space.append(int(bm.read_bits(bit_len)))
        rem_chars = rem_chars - sym_space[-1]
        bit_len = len(bin(rem_chars - 1)) - 2
        # print(sym_space, rem_chars, bit_len)
        # 0 is possible as the first character, but not as a spacing
        # if there are no special chars, the last number will be >= 66; 
        # but 0 may be used instead as the end after this
        if sym_space[-1] == 0 and len(sym_space) > 1: 
            break
    
    sym_space.pop() # remove the last, unused space, which ends the array
    print(sym_space)

    #print(len(sym_space))
    x1=x2=x3=False
    for i in range(1,len(sym_space)):
        sym_space[i] = sym_space[i] + sym_space[i-1]
        if sym_space[i] >= 48 and x1 == False: # digits
            x1 = True
            sym_space[i] += 10
        if sym_space[i] >= 65 and x2 == False: # uppercase letters
            x2 = True
            sym_space[i] += 26
        if sym_space[i] >= 97 and x3 == False: # lowercase layers
            x3 = True
            sym_space[i] += 26
    # now sym_space gives the ascii code for all the special characters

    print(sym_space)

    sym = []
    for i in sym_space:
        sym.append(chr(i))

    return sym

In [46]:
sim = remove_sym_arr(byte_count)
n = len(sim) + 26 + 10
size = len(bin(n)) - 2

ch = []
for i in range(48):
    if chr(i) in sim:    
        ch.append(chr(i))
ch = ch + num
for i in range(58, 97):
    if chr(i) in sim:    
        ch.append(chr(i))
ch = ch + low
for i in range(123, 128):
    if chr(i) in sim:
        ch.append(chr(i))
print(size, n, len(ch), sim)

[10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1]
[10, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 58, 59, 60, 62, 63, 91, 93, 124, 125]
6 61 61 ['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '>', '?', '[', ']', '|', '}']


In [ ]:
# ['\n', ' ', '!', '\"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '-', '.', '/' ]
# sim2 = [':', ';', '<', '>', '?', '[', ']']
# sim3 = ['|', '}']
# {10(7), 22(6), 1(6), 1(6), 2(5), 1(5), 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 3}
# size_sim_arr = [7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 2, 2]
#len([10, 22, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 5, 1, 0])

# Finding the Machines

In [47]:
# set up the alphabet
array_nums = [i for i in range(len(ch)+1)]
counts_dynamic_static = count_mng(alphabet_in=array_nums)
decode_dyn_st = arith_code_mng(counts_mngr=counts_dynamic_static, byte_m=byte_count)

In [48]:
# p2: {'ABC'} -- The first three characters of the document (index 0 - 60). 
context = [] 
#size = 6
for i in range(3):
    context.append(decode_dyn_st.remove_character())
print(context) # should be [55, 41, 52]

[55, 41, 52]


In [ ]:
# correct is 5, 22, 0
print(byte_count.ii, byte_count.jj, byte_count.byte)

5 22 0


In [50]:
# p3: {c2_non0\[1\]\[0\]+1, c2_non0\[1\]\[1:\]- c2_non0\[1\]\[0:-1\]} 
# two indicies, first of length 61, second of leng
# there should be 1427 numbers loaded, each with a length of 6 bits

p3_arr = ([], [])
notdone = True
i = 0 # i is increasing by one at the beginning -- this gives the correct results
j = 0
notFirst= False

while notdone:
    jo = j
    j = decode_dyn_st.remove_character()
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(ch):
            print('i greater than len')
            #byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(ch):
            j = decode_dyn_st.remove_character()
        else:
            notdone = False
    if notdone:
        p3_arr[0].append(i)
        p3_arr[1].append(j)
    notFirst = True

In [ ]:
print(len(p3_arr[0])) # 1427

1427


In [ ]:
# correct numbers: 1 1052, 80, 2
print(byte_count.ii, byte_count.jj, byte_count.byte, decode_dyn_st.bits) # should be 7 1051, not 1 1052, but bits is 2, so this is correct

1 1052 80


In [54]:
# p4: {c3n_non0\[1\]\[0\]+1, c3n_non0\[1\]\[1:\]- c3n_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive.
# y = 61
# target size -- 12699

p4_arr = ([], [])
notdone = True
i = 0
j = 0
notFirst= False

while notdone:
    jo = j
    j = decode_dyn_st.remove_character()
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(p3_arr[0]):
            print('i greater than len')
            #byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(p3_arr[0]):
            j = decode_dyn_st.remove_character()
        else:
            notdone = False
    if notdone:
        p4_arr[0].append(i)
        p4_arr[1].append(j)
    notFirst= True

In [ ]:
print(len(p4_arr[0]), len(set(p4_arr[0]))) # 12699, 1427

12699 1427


In [ ]:
print(byte_count.ii, byte_count.jj, byte_count.byte) # 3 9587 4

3 9587 4


In [ ]:
# so far correct

In [57]:
#  p5: {c4_non0\[1\]\[0\]+1, c4_non0\[1\]\[1:\]- c4_non0\[1\]\[0:-1\]} -- Add 61 to all non-positive character spacings. Counts are only important here, as the old counts can be calculated from these counts and the first 3 characters.
# z = 61, to indicate end of file 
# target size: 59088

p5_arr = ([], [])
notdone = True
i = 0
j = 0
notFirst= False

while notdone:
    jo = j
    j = decode_dyn_st.remove_character()
    if j <= jo and notFirst:
        i = i + 1
        if i >= len(p4_arr[0]):
            print('i greater than len')
            #byte_count.revert()
            notdone = False
    elif j >= len(ch):
        i = i + 1
        if i < len(p4_arr[0]):
            j = decode_dyn_st.remove_character()
        else:
            notdone = False
    if notdone:
        p5_arr[0].append(i)
        p5_arr[1].append(j)
    notFirst= True

In [83]:
print(len(p5_arr[0]), len(set(p5_arr[0])))

59088 12699


Everything is now correct, including the window. Lets look to see where I am in the byte stream.

In [ ]:
print(byte_count.ii, byte_count.jj, byte_count.byte, decode_dyn_st.bits) # 6 48644 2 0

6 48644 2


In [ ]:
decode_dyn_st.end_stream()
print(byte_count.ii, byte_count.jj, byte_count.byte) # shouldn't change anything

6 48644 2


It is starting at the correct place right now, right after the 39 zeros. Now I could either load the new alphabet first, or afterwords. Loading it first is probably better, but I will start with afterwords. 

Loading first means skipping to the start of the next byte, whereas I need to load 39 more zeros if it is afterwords. I will do the second version since this needs to be added to a big file. I could probably do it with less whitespace, but the amount of whitespace is actually not that large in total -- no more than 8 bytes here.

# Finding the Counts

In [ ]:
print(byte_count.ii, byte_count.jj, byte_count.byte, byte_count.ll, len(p5_arr[0])) # 6 48644 2 90640 59088

6 48644 2 90640 59088


We now have a dynamic alphabet. We will actually be retrieving indicies of the number in the alphabet to be decoded later. The alphabet will then give us the count.

In [68]:
# set up the alphabet
# we have a new count management, but the same byte management
counts_dynamic = count_mng()
decode_dyn = arith_code_mng(counts_mngr=counts_dynamic, byte_m=byte_count)

In [69]:
count4 = [] #np.zeros(len(p5_arr[0]), 'int')

# count4
for i in range(len(p5_arr[0])):
    count4.append(decode_dyn.remove_character())

#whitespace2 = byte_count.read_bits(3) # this exists only in the full file encod
#byte_count.to_end_of_byte() # should be the end now, but needed if the files are joined

In [76]:
decode_dyn.end_stream()

In [70]:
len(counts_dynamic.alphabet)

1476

In [71]:
print(counts_dynamic.counts)

[2603, 16878, 3838, 1091, 2014, 1552, 396, 235, 451, 1282, 6560, 236, 60, 85, 2, 4, 11, 8, 28, 28, 14, 3, 1, 64, 29, 29, 5, 4, 2, 9, 6, 2, 3, 9, 232, 2, 2, 2, 251, 117, 824, 178, 696, 889, 693, 588, 546, 456, 10, 296, 376, 116, 58, 6, 175, 25, 343, 22, 247, 99, 115, 305, 35, 159, 59, 59, 483, 191, 118, 134, 88, 2, 21, 133, 136, 147, 1, 17, 187, 103, 255, 54, 264, 20, 20, 18, 30, 52, 10, 37, 86, 175, 166, 89, 6, 34, 4, 102, 82, 62, 1, 96, 12, 13, 15, 60, 28, 36, 8, 31, 185, 56, 40, 326, 36, 15, 94, 59, 3, 167, 11, 2, 9, 98, 25, 51, 10, 42, 6, 70, 11, 105, 59, 115, 26, 20, 42, 70, 16, 85, 2, 132, 33, 43, 8, 2, 65, 6, 8, 72, 88, 62, 107, 28, 121, 73, 4, 41, 30, 29, 7, 14, 24, 26, 17, 13, 2, 73, 52, 1, 18, 21, 237, 75, 34, 7, 14, 59, 17, 25, 3, 8, 5, 32, 9, 25, 1, 1, 4, 2, 2, 7, 55, 82, 2, 1, 4, 2, 1, 1, 3, 10, 5, 16, 3, 2, 61, 2, 13, 2, 14, 27, 3, 1, 25, 21, 66, 7, 12, 11, 1, 12, 8, 10, 4, 2, 1, 3, 1, 9, 4, 22, 21, 1, 1, 1, 2, 3, 1, 19, 4, 1, 1, 17, 31, 1, 1, 13, 41, 4, 4, 1, 29, 3, 2, 1,

In [72]:
arr1 = np.array(counts_dynamic.counts)
arr2 = np.array([2603, 16878, 3838, 1091, 2014, 1552, 396, 235, 451, 1282, 6560, 236, 60, 85, 2, 4, 11, 8, 28, 28, 14, 3, 1, 64, 29, 29, 5, 4, 2, 9, 6, 2, 3, 9, 232, 2, 2, 2, 251, 117, 824, 178, 696, 889, 693, 588, 546, 456, 10, 296, 376, 116, 58, 6, 175, 25, 343, 22, 247, 99, 115, 305, 35, 159, 59, 59, 483, 191, 118, 134, 88, 2, 21, 133, 136, 147, 1, 17, 187, 103, 255, 54, 264, 20, 20, 18, 30, 52, 10, 37, 86, 175, 166, 89, 6, 34, 4, 102, 82, 62, 1, 96, 12, 13, 15, 60, 28, 36, 8, 31, 185, 56, 40, 326, 36, 15, 94, 59, 3, 167, 11, 2, 9, 98, 25, 51, 10, 42, 6, 70, 11, 105, 59, 115, 26, 20, 42, 70, 16, 85, 2, 132, 33, 43, 8, 2, 65, 6, 8, 72, 88, 62, 107, 28, 121, 73, 4, 41, 30, 29, 7, 14, 24, 26, 17, 13, 2, 73, 52, 1, 18, 21, 237, 75, 34, 7, 14, 59, 17, 25, 3, 8, 5, 32, 9, 25, 1, 1, 4, 2, 2, 7, 55, 82, 2, 1, 4, 2, 1, 1, 3, 10, 5, 16, 3, 2, 61, 2, 13, 2, 14, 27, 3, 1, 25, 21, 66, 7, 12, 11, 1, 12, 8, 10, 4, 2, 1, 3, 1, 9, 4, 22, 21, 1, 1, 1, 2, 3, 1, 19, 4, 1, 1, 17, 31, 1, 1, 13, 41, 4, 4, 1, 29, 3, 2, 1, 2, 6, 2, 4, 22, 13, 3, 14, 78, 80, 4, 6, 16, 47, 6, 39, 16, 29, 7, 4, 9, 3, 9, 8, 31, 1, 1, 46, 48, 6, 1, 6, 8, 9, 43, 4, 13, 2, 3, 19, 18, 10, 5, 3, 44, 4, 1, 1, 28, 5, 1, 5, 8, 10, 38, 13, 14, 8, 13, 17, 11, 5, 3, 49, 11, 27, 11, 15, 3, 5, 21, 9, 1, 5, 2, 54, 1, 3, 1, 4, 1, 15, 10, 4, 15, 7, 8, 39, 14, 10, 30, 3, 37, 1, 55, 5, 14, 1, 4, 1, 6, 7, 5, 1, 5, 4, 7, 3, 1, 14, 6, 5, 2, 11, 12, 5, 1, 3, 5, 1, 6, 18, 10, 29, 2, 25, 1, 2, 6, 2, 22, 4, 2, 1, 52, 2, 9, 3, 4, 12, 12, 1, 12, 1, 2, 3, 1, 1, 1, 1, 1, 3, 1, 17, 1, 1, 6, 1, 21, 30, 7, 2, 4, 18, 10, 1, 1, 1, 1, 2, 2, 4, 6, 4, 3, 4, 1, 1, 25, 1, 5, 1, 2, 19, 27, 1, 1, 43, 7, 16, 1, 9, 10, 3, 1, 1, 4, 1, 2, 4, 1, 8, 11, 13, 31, 53, 1, 5, 44, 9, 5, 2, 2, 1, 12, 6, 10, 14, 1, 1, 13, 4, 10, 5, 1, 5, 3, 1, 1, 11, 2, 3, 2, 2, 5, 5, 1, 4, 1, 1, 2, 28, 4, 1, 11, 3, 2, 24, 1, 5, 1, 4, 3, 2, 10, 1, 5, 19, 1, 5, 6, 4, 5, 7, 1, 1, 31, 1, 2, 26, 1, 8, 1, 3, 21, 1, 2, 24, 15, 1, 1, 1, 1, 2, 11, 24, 1, 9, 1, 1, 15, 2, 2, 9, 1, 13, 2, 2, 1, 6, 42, 3, 6, 1, 4, 38, 3, 1, 2, 27, 1, 1, 3, 37, 2, 21, 3, 20, 8, 7, 2, 17, 5, 5, 1, 2, 10, 2, 1, 2, 1, 10, 1, 4, 5, 4, 10, 2, 6, 5, 1, 13, 1, 2, 2, 7, 2, 13, 1, 1, 1, 1, 6, 14, 2, 1, 73, 8, 1, 2, 11, 6, 2, 1, 1, 1, 1, 1, 2, 1, 1, 5, 1, 3, 2, 3, 4, 1, 12, 1, 21, 4, 1, 3, 27, 9, 1, 8, 1, 4, 12, 1, 1, 4, 5, 1, 1, 1, 4, 1, 1, 4, 1, 1, 34, 9, 7, 1, 2, 1, 2, 2, 11, 3, 10, 4, 7, 41, 6, 14, 14, 2, 6, 16, 5, 2, 2, 2, 1, 2, 6, 15, 6, 4, 2, 5, 7, 7, 1, 9, 3, 16, 1, 1, 1, 2, 1, 1, 1, 32, 1, 2, 26, 21, 3, 5, 6, 3, 1, 3, 15, 2, 1, 5, 5, 1, 23, 5, 9, 7, 7, 8, 2, 3, 4, 1, 15, 8, 4, 1, 1, 1, 14, 16, 2, 4, 15, 1, 15, 1, 1, 3, 2, 3, 3, 4, 1, 3, 1, 8, 2, 2, 5, 5, 2, 5, 11, 13, 5, 1, 2, 1, 1, 10, 1, 3, 4, 1, 6, 3, 3, 2, 1, 3, 4, 1, 5, 2, 4, 1, 3, 8, 3, 6, 6, 4, 11, 1, 4, 1, 14, 1, 5, 10, 5, 1, 2, 2, 2, 2, 1, 6, 3, 1, 2, 1, 8, 2, 6, 3, 1, 1, 3, 6, 3, 16, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 3, 2, 6, 5, 2, 7, 4, 2, 1, 1, 1, 3, 14, 3, 1, 1, 2, 1, 4, 5, 1, 4, 3, 1, 1, 5, 6, 1, 13, 2, 1, 6, 3, 2, 1, 4, 5, 1, 1, 6, 1, 1, 2, 8, 3, 7, 1, 3, 1, 1, 4, 2, 2, 1, 1, 11, 3, 9, 3, 4, 3, 3, 8, 1, 4, 1, 7, 3, 6, 2, 3, 4, 4, 2, 1, 3, 1, 3, 2, 2, 2, 5, 1, 3, 1, 2, 2, 1, 1, 1, 1, 2, 13, 5, 1, 1, 2, 1, 1, 1, 9, 1, 2, 2, 2, 1, 2, 5, 3, 1, 2, 2, 1, 4, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 9, 1, 2, 1, 1, 2, 3, 1, 2, 1, 4, 1, 1, 1, 2, 2, 3, 1, 3, 5, 1, 2, 1, 1, 5, 1, 2, 1, 1, 3, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 3, 1, 5, 1, 2, 2, 5, 2, 1, 1, 1, 3, 5, 6, 4, 3, 3, 1, 2, 1, 1, 4, 1, 1, 2, 3, 1, 2, 3, 1, 1, 1, 2, 2, 4, 1, 7, 1, 4, 2, 2, 1, 1, 2, 1, 1, 4, 2, 4, 3, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 5, 1, 2, 1, 1, 1, 3, 6, 2, 1, 3, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 2, 1, 3, 1, 1, 1, 4, 2, 1, 2, 1, 2, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 4, 2, 1, 1, 5, 2, 3, 1, 1, 1, 1, 5, 1, 1, 1, 2, 4, 1, 5, 1, 1, 1, 1, 8, 5, 1, 1, 4, 3, 6, 2, 2, 4, 1, 1, 1, 4, 1, 2, 1, 1, 1, 1, 4, 4, 3, 3, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 3, 1, 1, 2, 2, 1, 1, 4, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 1, 3, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [73]:
print(np.nonzero(arr1-arr2))

(array([], dtype=int64),)


In [ ]:
print(byte_count.ii, byte_count.jj, byte_count.byte, byte_count.ll, decode_dyn.bits) # 3 88080 8 90640 2

3 88080 8 90640


# Getting the Counts

Below is the correct starting point. Counts are correct. I will figure out why I got the wrong starting point.

In [ ]:
print(byte_count.ii, byte_count.jj, byte_count.byte, byte_count.ll) # 3 88080 8 90640
tar_len = len(counts_dynamic.alphabet)

3 88080 8 90640


In [79]:
count_alphabet = []
for i in range(tar_len):
    count_alphabet.append(byte_count.read_ternary())

In [80]:
print(count_alphabet)

[4, 1, 3, 8, 5, 6, 18, 24, 17, 7, 2, 32, 98, 71, 1066, 336, 276, 441, 151, 180, 221, 791, 2179, 78, 134, 169, 371, 876, 706, 281, 443, 718, 1316, 234, 30, 1305, 1213, 1072, 25, 49, 10, 40, 12, 9, 11, 13, 14, 16, 248, 23, 19, 54, 95, 279, 34, 178, 20, 184, 26, 51, 48, 22, 147, 45, 84, 82, 15, 35, 42, 41, 57, 1033, 160, 46, 44, 43, 2247, 224, 31, 53, 29, 105, 27, 201, 213, 185, 159, 106, 263, 165, 64, 37, 36, 68, 368, 156, 775, 58, 59, 87, 1186, 52, 290, 269, 188, 86, 141, 124, 326, 128, 33, 96, 99, 21, 116, 245, 62, 102, 350, 39, 192, 1017, 396, 65, 166, 107, 327, 92, 776, 73, 297, 55, 80, 61, 176, 187, 115, 76, 278, 56, 1160, 38, 114, 100, 393, 679, 85, 536, 647, 60, 63, 81, 50, 144, 47, 79, 841, 117, 140, 139, 417, 240, 146, 161, 197, 286, 720, 69, 122, 962, 204, 191, 28, 67, 154, 644, 254, 74, 177, 157, 575, 231, 471, 138, 369, 170, 1525, 1491, 586, 1077, 898, 493, 93, 70, 1698, 1810, 430, 1277, 707, 2226, 691, 230, 627, 237, 650, 2803, 91, 577, 262, 1318, 214, 155, 764, 1042, 152, 1

In [81]:
arr1 = np.array(count_alphabet)
arr2 = [4, 1, 3, 8, 5, 6, 18, 24, 17, 7, 2, 32, 98, 71, 1066, 336, 276, 441, 151, 180, 221, 791, 2179, 78, 134, 169, 371, 876, 706, 281, 443, 718, 1316, 234, 30, 1305, 1213, 1072, 25, 49, 10, 40, 12, 9, 11, 13, 14, 16, 248, 23, 19, 54, 95, 279, 34, 178, 20, 184, 26, 51, 48, 22, 147, 45, 84, 82, 15, 35, 42, 41, 57, 1033, 160, 46, 44, 43, 2247, 224, 31, 53, 29, 105, 27, 201, 213, 185, 159, 106, 263, 165, 64, 37, 36, 68, 368, 156, 775, 58, 59, 87, 1186, 52, 290, 269, 188, 86, 141, 124, 326, 128, 33, 96, 99, 21, 116, 245, 62, 102, 350, 39, 192, 1017, 396, 65, 166, 107, 327, 92, 776, 73, 297, 55, 80, 61, 176, 187, 115, 76, 278, 56, 1160, 38, 114, 100, 393, 679, 85, 536, 647, 60, 63, 81, 50, 144, 47, 79, 841, 117, 140, 139, 417, 240, 146, 161, 197, 286, 720, 69, 122, 962, 204, 191, 28, 67, 154, 644, 254, 74, 177, 157, 575, 231, 471, 138, 369, 170, 1525, 1491, 586, 1077, 898, 493, 93, 70, 1698, 1810, 430, 1277, 707, 2226, 691, 230, 627, 237, 650, 2803, 91, 577, 262, 1318, 214, 155, 764, 1042, 152, 158, 94, 573, 264, 256, 1763, 292, 553, 296, 402, 1867, 15677, 495, 2901, 212, 1260, 142, 183, 3403, 1391, 1501, 685, 654, 4606, 210, 569, 4327, 986, 167, 118, 955, 2119, 289, 132, 458, 662, 4083, 133, 477, 693, 1487, 1294, 535, 763, 633, 203, 329, 840, 261, 75, 66, 850, 622, 194, 119, 422, 112, 232, 200, 253, 437, 465, 714, 364, 345, 137, 5346, 1409, 108, 97, 312, 4236, 372, 392, 300, 101, 335, 375, 1344, 911, 209, 242, 361, 559, 862, 103, 529, 2836, 3805, 129, 530, 2704, 462, 229, 324, 110, 359, 268, 351, 243, 218, 308, 678, 548, 89, 331, 126, 226, 198, 842, 454, 193, 405, 2068, 611, 1216, 83, 2160, 1618, 3070, 508, 1460, 206, 283, 623, 244, 347, 466, 104, 172, 401, 171, 728, 127, 2230, 111, 513, 271, 1011, 377, 1127, 362, 540, 464, 1090, 445, 568, 321, 1375, 1224, 236, 552, 452, 651, 322, 225, 380, 6617, 676, 407, 3652, 459, 196, 238, 143, 646, 162, 2548, 1020, 460, 670, 233, 1301, 941, 1446, 77, 1043, 259, 716, 413, 235, 316, 1458, 356, 2193, 967, 773, 1331, 5078, 5191, 1433, 2507, 880, 5513, 266, 3708, 5331, 384, 1046, 257, 163, 360, 1502, 476, 247, 260, 760, 2423, 2241, 606, 631, 1328, 560, 457, 781, 397, 742, 2021, 1580, 190, 1923, 673, 2476, 957, 181, 149, 797, 3036, 113, 330, 252, 8114, 381, 365, 660, 2737, 5255, 867, 7226, 1684, 541, 1772, 388, 288, 208, 123, 90, 3035, 522, 88, 399, 333, 2748, 908, 1532, 295, 436, 291, 164, 2210, 945, 311, 488, 280, 616, 2553, 363, 927, 643, 3741, 217, 814, 603, 1526, 1214, 684, 343, 905, 704, 3158, 1346, 944, 153, 655, 787, 228, 1075, 1611, 189, 1170, 543, 1756, 410, 597, 1040, 338, 5776, 520, 175, 1272, 431, 652, 526, 1530, 403, 4842, 1253, 130, 13470, 1572, 168, 4764, 451, 2277, 878, 195, 1688, 1067, 125, 202, 1348, 983, 3160, 1514, 473, 227, 207, 1181, 303, 913, 1152, 220, 1577, 815, 273, 1105, 255, 1508, 566, 2608, 505, 109, 712, 479, 3264, 628, 136, 601, 1191, 1408, 150, 1997, 2062, 444, 131, 909, 179, 695, 186, 382, 587, 726, 215, 340, 827, 2401, 754, 447, 1207, 3256, 901, 2730, 306, 790, 528, 318, 591, 239, 874, 456, 500, 1283, 174, 3368, 1076, 645, 307, 1124, 274, 2779, 1482, 1059, 1481, 663, 250, 1868, 1770, 72, 293, 2026, 749, 328, 285, 948, 11638, 42912, 7243, 1822, 1718, 1422, 21975, 830, 579, 1257, 625, 1028, 1230, 837, 915, 319, 2458, 145, 873, 1431, 668, 148, 391, 1899, 414, 4307, 812, 282, 1113, 2811, 836, 774, 3039, 2531, 1623, 823, 1293, 5743, 426, 2446, 2434, 121, 383, 320, 11341, 951, 9689, 1107, 1555, 272, 682, 440, 996, 408, 120, 341, 246, 216, 918, 636, 223, 416, 1015, 1095, 1259, 1860, 1566, 574, 305, 314, 472, 539, 497, 309, 310, 999, 609, 612, 251, 891, 863, 1117, 1657, 1221, 2991, 3238, 135, 1615, 649, 199, 173, 698, 373, 358, 504, 2866, 879, 211, 635, 3481, 354, 398, 782, 182, 509, 277, 294, 455, 385, 1285, 786, 510, 1099, 219, 517, 567, 5381, 1381, 1101, 241, 267, 1320, 519, 222, 3982, 275, 732, 10710, 624, 751, 705, 659, 480, 2075, 743, 5364, 265, 721, 739, 390, 374, 638, 302, 325, 287, 355, 1708, 1136, 2711, 1896, 284, 2320, 865, 434, 959, 639, 715, 730, 819, 1053, 800, 607, 1393, 518, 483, 669, 2358, 1108, 323, 516, 332, 686, 594, 378, 2214, 734, 3644, 304, 1369, 353, 342, 475, 946, 580, 759, 900, 1252, 788, 412, 1279, 17485, 772, 2176, 376, 703, 576, 710, 1774, 1906, 1429, 379, 435, 249, 1085, 1096, 3235, 572, 596, 486, 681, 1503, 688, 746, 907, 1023, 758, 429, 1405, 386, 313, 780, 205, 690, 984, 1097, 1512, 2335, 463, 270, 1058, 2958, 1407, 640, 1274, 349, 507, 953, 801, 615, 1044, 2484, 619, 558, 851, 299, 912, 4905, 450, 490, 834, 583, 557, 499, 942, 2163, 337, 1086, 1368, 1313, 427, 339, 542, 1933, 395, 2714, 5167, 404, 626, 822, 3031, 1256, 482, 756, 406, 904, 489, 352, 700, 301, 584, 442, 769, 433, 549, 474, 1306, 453, 534, 394, 1080, 1854, 409, 1149, 590, 621, 970, 1159, 370, 1735, 461, 1103, 1200, 877, 1513, 792, 563, 1041, 778, 258, 571, 5430, 4094, 511, 503, 1092, 859, 389, 811, 1343, 816, 1082, 1945, 2257, 485, 600, 1696, 561, 491, 1475, 554, 2694, 711, 2927, 1336, 1816, 582, 2503, 1032, 1629, 618, 701, 1049, 1672, 869, 1164, 894, 1700, 1746, 856, 637, 1271, 2398, 1541, 1094, 1142, 1060, 789, 1435, 3553, 1493, 608, 672, 470, 860, 872, 2000, 1455, 803, 820, 994, 1413, 1325, 565, 1658, 7639, 4466, 2585, 1439, 989, 3396, 717, 438, 1374, 1112, 1561, 1758, 418, 752, 1534, 1258, 544, 689, 1070, 1667, 902, 496, 1247, 831, 4563, 602, 1168, 1183, 2822, 1353, 858, 964, 1228, 448, 833, 1128, 1178, 425, 1055, 804, 991, 1175, 387, 420, 357, 733, 546, 1110, 550, 919, 1787, 868, 667, 1291, 1073, 940, 570, 736, 3194, 1121, 954, 1643, 4066, 683, 532, 346, 1679, 419, 2078, 1052, 980, 950, 1037, 1527, 1123, 1352, 1248, 317, 593, 334, 1536, 2191, 961, 428, 657, 4977, 938, 848, 838, 4608, 1546, 1134, 3000, 468, 727, 533, 828, 766, 2845, 2708, 1130, 699, 315, 1902, 4277, 692, 929, 1327, 656, 713, 1985, 1268, 2017, 589, 992, 1120, 11256, 4831, 1833, 2134, 3111, 2008, 2297, 1938, 1141, 501, 2372, 1989, 1929, 3188, 4899, 2672, 2871, 1862, 1767, 498, 1244, 556, 1479, 3447, 6625, 5620, 2397, 2150, 432, 5166, 2533, 8013, 494, 1239, 1521, 1871, 777, 1802, 1974, 1184, 702, 1163, 1499, 687, 2963, 1138, 1083, 630, 1235, 629, 610, 1036, 889, 641, 762, 6424, 1019, 2565, 972, 1622, 671, 3151, 1365, 747, 24084, 1370, 2686, 1034, 2108, 3832, 982, 725, 1447, 771, 523, 613, 492, 1223, 1402, 449, 832, 738, 1310, 4946, 886, 1498, 367, 3165, 897, 914, 1276, 892, 1192, 481, 3022, 2738, 735, 795, 348, 400, 1047, 1702, 737, 1003, 344, 664, 947, 411, 1524, 2703, 722, 415, 977, 515, 642, 3213, 3764, 1143, 527, 366, 709, 675, 847, 753, 956, 578, 846, 1317, 1093, 478, 1448, 1226, 1241, 2863, 1329, 821, 592, 512, 1607, 439, 298, 1410, 1773, 484, 599, 985, 799, 783, 524, 6446, 1366, 1139, 9567, 2082, 1900, 694, 1520, 2904, 1150, 620, 4010, 757, 829, 1148, 1292, 3584, 924, 2232, 1890, 5872, 1436, 1722, 521, 2046, 4020, 1031, 731, 2337, 839, 2425, 487, 446, 895, 1522, 719, 805, 598, 3912, 952, 779, 4384, 1872, 870, 2086, 1189, 1377, 1270, 581, 2667, 3054, 1220, 3727, 969, 1002, 1668, 922, 1754, 2696, 1187, 1218, 910, 2430, 545, 741, 661, 740, 981, 3286, 5892, 2884, 3614, 5408, 2117, 993, 1634, 1683, 537, 604, 1211, 423, 853, 421, 1456, 1275, 813, 467, 3051, 1264, 469, 882, 2060, 1963, 793, 2590, 4392, 729, 2894, 2135, 666, 1071, 906, 2984, 2734, 665, 1225, 748, 1022, 887, 1079, 2199, 966, 2069, 1281, 824, 562, 1467, 1864, 1255, 4826, 2789, 937, 806, 1116, 845, 1177, 949, 3225, 1576, 1950, 1154, 2692, 3477, 677, 864, 1434, 1406, 1616, 1761, 585, 1165, 1169, 2047, 1682, 1219, 2218, 1459, 3704, 506, 1379, 933, 8287, 3208, 1834, 723, 1290, 1451, 3243, 1131, 1025, 1843, 849, 2523, 3616, 525, 1670, 1715, 10628, 32681, 2402, 2251, 6895, 3266, 7329, 3452, 5403, 871, 1666, 1904, 796, 1210, 1956, 1819, 1970, 4178, 617, 1063, 765, 4475, 1492, 2992, 2680, 2355, 1236, 1401, 2361, 1392, 809, 1185, 1823, 3578, 1383, 1282, 1001, 1308, 3817, 1599, 1284, 5783, 2497, 1005, 680, 1725, 514, 744, 7869, 2022, 653]
arr2 = np.array(arr2)
print(np.nonzero(arr1-arr2))

(array([], dtype=int64),)


All correct.

# Forming the Count Array

All the data has been loaded sucessfully. The lists of contexts are identical to the old format, but the counts are not quite in the required format for decoding the text like before.

In [87]:
len(count_alphabet), len(set(count4))

(1476, 1476)

In [88]:
for i in range(len(count4)):
    count4[i] = count_alphabet[count4[i]]

In [ ]:
print(count4[0:30], count4[-30:])
# [4, 1, 4, 1, 3, 4, 3, 8, 5, 6, 18, 3, 24, 17, 1, 7, 6, 8, 8, 2, 1, 8, 2, 2, 1, 32, 98, 71, 1, 1] 
# [4, 8, 1, 1, 17, 1, 1, 2, 1, 5, 1, 4, 1, 1, 2, 1, 4, 7, 1, 110, 8, 2, 2, 6, 1, 1, 1, 2, 1, 1]

[4, 1, 4, 1, 3, 4, 3, 8, 5, 6, 18, 3, 24, 17, 1, 7, 6, 8, 8, 2, 1, 8, 2, 2, 1, 32, 98, 71, 1, 1] [4, 8, 1, 1, 17, 1, 1, 2, 1, 5, 1, 4, 1, 1, 2, 1, 4, 7, 1, 110, 8, 2, 2, 6, 1, 1, 1, 2, 1, 1]


Correct.